In [ ]:
from __future__ import print_function
import numpy as np
!pip install efficientnet_pytorch
!pip install timm

import pandas as pd
from google.colab import drive
import torch
import torch.nn as nn
import os
from efficientnet_pytorch import EfficientNet
import torch.optim as optim
import glob
from itertools import chain
import os
import random
import zipfile
import matplotlib.pyplot as plt
import torch.nn.functional as F

from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm



drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.1 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from torchvision import transforms
import pandas as pd
import os

def load_data(data_file, data_dir='/content/drive/Shareddrives/IC - Datasets/dataset/lens/',
              img_ext='.bmp', img_shape=(224, 224)):

    class_mapper = {'Colored': 0, 'Normal': 1, 'Transparent': 2}
    data = []
    labels = []

    file_content = pd.read_csv(data_file, skiprows=[0], sep=',')
    len_file = len(file_content)

    transform = transforms.Compose([
        transforms.RandomRotation(10),  # Rotação aleatória de até 10 graus
        transforms.RandomHorizontalFlip(),  # Espelhamento horizontal aleatório
        transforms.RandomVerticalFlip(),  # Espelhamento vertical aleatório
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Ajuste aleatório de brilho, contraste, saturação e tom
        transforms.RandomResizedCrop(input_shape[0]),  # Recorte aleatório redimensionado para o tamanho de entrada
        transforms.ToTensor(),  # Converter a imagem em tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizar os valores dos canais RGB
    ])

    for e, row in file_content.iterrows():
        img_path = os.path.join(data_dir, row[0]) + img_ext
        image = Image.open(img_path).convert('RGB')
        image = transform(image)
        label = class_mapper[row[1]]

        data.append(image)
        labels.append(label)

    data = torch.stack(data)
    labels = torch.tensor(labels)

    return data, labels

In [ ]:
data_dir = '/content/drive/Shareddrives/IC - Datasets/dataset/lens/'
train_data_file = os.path.join(data_dir, 'CogentAnnotationTrain.csv')
test_data_file = os.path.join(data_dir, 'CogentAnnotationTest.csv')

num_classes = 3
input_shape = (224, 224, 3)
# Parâmetros de treinamento
learning_rate = 0.001
batch_size = 16
epochs = 10

x_train, y_train = load_data(train_data_file, data_dir=data_dir, img_shape=input_shape)
x_test, y_test = load_data(test_data_file, data_dir=data_dir, img_shape=input_shape)

train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(x_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

x_train shape: torch.Size([1752, 3, 224, 224]) - y_train shape: torch.Size([1752])
x_test shape: torch.Size([1754, 3, 224, 224]) - y_test shape: torch.Size([1754])


In [ ]:
import timm
import torchvision.transforms as transforms


# Definir o modelo ViT e o extrator de características
model = timm.create_model('vit_base_patch16_224', pretrained=True)  # Carregar o modelo ViT pré-treinado
transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224)])  # Pré-processar as imagens

class ViTCNN(nn.Module):
    def __init__(self):
        super(ViTCNN, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)

    def forward(self, x):
        outputs = self.vit(x)
        return outputs

model = ViTCNN()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def accuracy(outputs, labels):
    _, predicted = torch.max(outputs, dim=1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    return correct / total

def train(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        running_accuracy = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)  # Mover os dados de entrada para a GPU
            labels = labels.to(device)  # Mover os rótulos para a GPU
            optimizer.zero_grad()
            inputs = transform(inputs)  # Pré-processar as imagens usando o transform
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_accuracy += accuracy(outputs, labels)
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = running_accuracy / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.4f}")

train(model, train_loader, criterion, optimizer, 10)


Epoch 1/10 - Loss: 1.5687 - Accuracy: 0.3205
Epoch 2/10 - Loss: 1.2044 - Accuracy: 0.3290
Epoch 3/10 - Loss: 1.1327 - Accuracy: 0.3517
Epoch 4/10 - Loss: 1.1126 - Accuracy: 0.3557
Epoch 5/10 - Loss: 1.1120 - Accuracy: 0.3545
Epoch 6/10 - Loss: 1.1058 - Accuracy: 0.3693
Epoch 7/10 - Loss: 1.1128 - Accuracy: 0.3585
Epoch 8/10 - Loss: 1.1099 - Accuracy: 0.3727
Epoch 9/10 - Loss: 1.0998 - Accuracy: 0.3750
Epoch 10/10 - Loss: 1.1088 - Accuracy: 0.3619


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Definir o modelo
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 56 * 56, 128),  # Corrigir o tamanho da camada de entrada
            nn.ReLU(),
            nn.Linear(128, 3)  # Alterar o número de classes para 3
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

def accuracy(outputs, labels):
    _, predicted = torch.max(outputs, dim=1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    return correct / total


# Definir o diretório dos dados
data_dir = '/content/drive/Shareddrives/IC - Datasets/dataset/lens/'
train_data_file = os.path.join(data_dir, 'CogentAnnotationTrain.csv')
test_data_file = os.path.join(data_dir, 'CogentAnnotationTest.csv')



# Criar instância do modelo
model =  EfficientNet.from_pretrained('efficientnet-b3')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Definir função de perda e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



# Função de treinamento
def train(model, train_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        running_accuracy = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)  # Mover os dados de entrada para a GPU
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_accuracy += accuracy(outputs, labels)
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = running_accuracy / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.4f}")




# Treinar o modelo
train(model, train_loader, criterion, optimizer, 40)


Loaded pretrained weights for efficientnet-b3
Epoch 1/40 - Loss: 1.1850 - Accuracy: 0.5602
Epoch 2/40 - Loss: 0.6265 - Accuracy: 0.6943
Epoch 3/40 - Loss: 0.5344 - Accuracy: 0.7597
Epoch 4/40 - Loss: 0.4093 - Accuracy: 0.8250
Epoch 5/40 - Loss: 0.3707 - Accuracy: 0.8517
Epoch 6/40 - Loss: 0.3321 - Accuracy: 0.8773
Epoch 7/40 - Loss: 0.2558 - Accuracy: 0.9017
Epoch 8/40 - Loss: 0.2268 - Accuracy: 0.9040
Epoch 9/40 - Loss: 0.2079 - Accuracy: 0.9239
Epoch 10/40 - Loss: 0.2126 - Accuracy: 0.9176
Epoch 11/40 - Loss: 0.1859 - Accuracy: 0.9301
Epoch 12/40 - Loss: 0.1374 - Accuracy: 0.9477
Epoch 13/40 - Loss: 0.1664 - Accuracy: 0.9386
Epoch 14/40 - Loss: 0.1392 - Accuracy: 0.9523
Epoch 15/40 - Loss: 0.1107 - Accuracy: 0.9619
Epoch 16/40 - Loss: 0.0939 - Accuracy: 0.9665
Epoch 17/40 - Loss: 0.1034 - Accuracy: 0.9614
Epoch 18/40 - Loss: 0.0996 - Accuracy: 0.9625
Epoch 19/40 - Loss: 0.1437 - Accuracy: 0.9449
Epoch 20/40 - Loss: 0.0777 - Accuracy: 0.9716
Epoch 21/40 - Loss: 0.0611 - Accuracy: 0.97

In [ ]:
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)  # Mover os dados de entrada para a GPU
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            running_accuracy += accuracy(outputs, labels)
    test_loss = running_loss / len(test_loader)
    test_accuracy = running_accuracy / len(test_loader)
    print(f"Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.4f}")




# Testar o modelo
test(model, test_loader, criterion)

Test Loss: 1.4564 - Test Accuracy: 0.6791


# Nova seção